# Lab 4: Build Multi AI Agent 
 
### Use-case: plan and write a blog post using watsonx.ai and crewai

In [2]:
!pip install crewai==0.80.0 crewai_tools==0.14.0 | tail -n 1

[documentation](https://docs.crewai.com/introduction)

- You may need to re-start your kernel after the last pip installls

In [3]:
# Warning control
import warnings
import os
from dotenv import load_dotenv
load_dotenv('.env')
warnings.filterwarnings('ignore')

- Imports

In [4]:
from crewai import Agent, Task, Crew
from crewai import LLM
from langchain.tools import tool
from typing import Any, List, Mapping, Optional

- Connect to IBM watsonx model

[watsonx and crewai integration](https://www.ibm.com/watsonx/developer/get-started/libraries/#crewai)

[invoke a model suitable for too calling](https://www.ibm.com/watsonx/developer/capabilities/tool-calling)

In [5]:
WATSONX_MODEL_ID = "watsonx/mistralai/mistral-large"
#MODEL=watsonx/meta-llama/llama-3-405b-instruct
#MODEL=watsonx/meta-llama/llama-3-1-70b-instruct

- get your api key, project_id

In [6]:
os.environ['WATSONX_URL'] = 'https://us-south.ml.cloud.ibm.com'
os.environ['PROJECT_ID'] = '4f6edd25-9060-4a3f-a906-b3915545a5a9'
os.environ['WATSONX_APIKEY'] = '7j4m2zAuBZYdelHTzFz5OC0pwRtI09wz7BHsj_mc7Jse'

In [7]:
custom_llm = LLM(
    model=WATSONX_MODEL_ID,
    base_url=os.getenv('WATSONX_URL'),
    project_id=os.getenv('PROJECT_ID'),
    max_tokens=10000,
    temperature=0.7
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [8]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=custom_llm
)

### Agent: Writer

In [9]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=custom_llm
)

In [10]:
my_str ="You are an editor who receives a blog post "\
              "from the Content Writer. "\
              "Your goal is to review the blog post "\
              "to ensure that it follows journalistic best practices,"\
              "provides balanced viewpoints "\
              "when providing opinions or assertions, "\
              "and also avoids major controversial topics "\
              "or opinions when possible."
my_str

'You are an editor who receives a blog post from the Content Writer. Your goal is to review the blog post to ensure that it follows journalistic best practices,provides balanced viewpoints when providing opinions or assertions, and also avoids major controversial topics or opinions when possible.'

### Agent: Editor

In [11]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=custom_llm
)

In [12]:
print(editor)

id=UUID('15a75ff2-d16a-4940-897c-5e0c60817021') formatting_errors=0 role='Editor' goal='Edit a given blog post to align with the writing style of the organization. ' backstory='You are an editor who receives a blog post from the Content Writer. Your goal is to review the blog post to ensure that it follows journalistic best practices,provides balanced viewpoints when providing opinions or assertions, and also avoids major controversial topics or opinions when possible.' config=None cache=True verbose=True max_rpm=None allow_delegation=False tools=[] max_iter=20 agent_executor=<crewai.agents.crew_agent_executor.CrewAgentExecutor object at 0x7fb87ec3ac50> llm=<crewai.llm.LLM object at 0x7fb87fc5e490> crew=None i18n=I18N(prompt_file=None) cache_handler=CacheHandler() tools_handler=<crewai.agents.tools_handler.ToolsHandler object at 0x7fb87ec3a050> max_tokens=None max_execution_time=None agent_ops_agent_name='Editor' agent_ops_agent_id=None step_callback=None use_system_prompt=True functio

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [13]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [14]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [15]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=True` allows you to see all the logs of the execution. 

In [16]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what your colleague gets.

In [17]:
mytopic = "IBM Concert Best Practices"
result = crew.kickoff(inputs={"topic": mytopic})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on IBM Concert Best Practices.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
---

### Content Plan for IBM Concert Best Practices

#### 1. Prioritize the Latest Trends, Key Players, and Noteworthy News

- **Latest Trends:**
  - Integration with AI and machine learning for enhanced automation.
  - Emphasis on cloud-based solutions for scalability and flexibility.
  - Focus on user experience and interface design.
  - Increased security measures and compliance with industry standards.

- **Key Players:**
  - IBM (primary developer and provider of IBM Concert).
  - Partners and integrators who work closely with IBM.
  - Industry leaders adopting IBM Concert for pro

- Display the results of your execution as markdown in the notebook.

In [19]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# Mastering IBM Concert: Best Practices for Optimal Project Management

In the rapidly evolving landscape of project management, tools like IBM Concert have become indispensable. IBM Concert offers a comprehensive suite of features designed to streamline workflows, enhance collaboration, and boost productivity. This article delves into the best practices for leveraging IBM Concert to its fullest potential, ensuring your projects run smoothly and efficiently.

## Understanding IBM Concert

IBM Concert is a robust project management solution designed to meet the diverse needs of modern businesses. Developed by IBM, this tool has set a benchmark in the industry with its advanced features and seamless integration capabilities.

**Definition and History:**
IBM Concert is a cloud-based project management platform that combines traditional project management tools with cutting-edge technology. It has evolved significantly since its inception, incorporating the latest trends in AI, machine learning, and user experience design.

**Key Features and Benefits:**
- **Enhanced Automation:** IBM Concert leverages AI and machine learning to automate repetitive tasks, freeing up valuable time for project teams.
- **Scalability and Flexibility:** Its cloud-based infrastructure allows for easy scaling, making it suitable for businesses of all sizes.
- **User-Friendly Interface:** The platform is designed with user experience in mind, ensuring that even non-technical users can navigate it effortlessly.

**How It Stands Out:**
IBM Concert differentiates itself through its strong emphasis on security and compliance, making it a top choice for industries with stringent regulatory requirements. Additionally, its seamless integration with other business applications ensures a cohesive work environment.

## Best Practices for Implementation

Implementing IBM Concert effectively requires a strategic approach. Here are some best practices to ensure a smooth transition:

**Planning and Setting Up IBM Concert:**
Before rolling out IBM Concert, it is crucial to plan thoroughly. Identify the key stakeholders, define roles and responsibilities, and set clear objectives for the implementation process.

**Customizing Workflows and Dashboards:**
IBM Concert allows for extensive customization. Tailor workflows and dashboards to align with your team's specific needs. This ensures that the tool is not just functional but also efficient in addressing your project management challenges.

**Training and Onboarding Team Members:**
Proper training is essential for the successful adoption of any new tool. Conduct comprehensive training sessions and provide ongoing support to help your team get accustomed to IBM Concert.

## Integration with AI and Machine Learning

AI and machine learning are revolutionizing project management. IBM Concert is at the forefront of this transformation.

**How AI Enhances Automation:**
AI can automate routine tasks such as data entry, reporting, and task assignments. This not only saves time but also reduces the risk of human error.

**Examples of AI Applications in IBM Concert:**
- **Predictive Analytics:** IBM Concert uses AI to predict project outcomes and identify potential risks.
- **Natural Language Processing:** The tool can understand and respond to natural language queries, making it more intuitive to use.

**Benefits and Challenges:**
While AI offers numerous benefits, it also presents challenges such as data privacy concerns and the need for continuous training of AI models. Balancing these aspects is key to successful AI integration.

## Cloud-Based Solutions

Cloud deployment offers significant advantages for project management.

**Advantages of Cloud Deployment:**
- **Scalability:** Cloud solutions can scale up or down based on demand, ensuring that you only pay for what you use.
- **Flexibility:** Access your projects from anywhere, at any time, with cloud-based solutions.

**Scalability and Flexibility:**
IBM Concert's cloud infrastructure ensures that your project management system can grow with your business. Whether you're a small startup or a large enterprise, IBM Concert can adapt to your needs.

**Security Measures in Cloud Environments:**
Security is a top concern in cloud environments. IBM Concert employs robust security measures, including data encryption and multi-factor authentication, to protect your sensitive information.

## User Experience and Interface Design

A user-friendly interface can make or break the adoption of a project management tool.

**Importance of a User-Friendly Interface:**
A well-designed interface enhances user engagement and productivity. IBM Concert focuses on providing a seamless and intuitive user experience.

**Best Practices for Interface Customization:**
- **Personalize Dashboards:** Allow team members to customize their dashboards to display the information that is most relevant to them.
- **Simplify Navigation:** Ensure that navigation is straightforward and intuitive, making it easy for users to find what they need.

**User Feedback and Continuous Improvement:**
Regularly collect user feedback to identify areas for improvement. Continuous improvement based on user insights helps keep the platform relevant and effective.

## Security and Compliance

Ensuring data security and compliance with industry standards is non-negotiable.

**Ensuring Data Security:**
IBM Concert employs advanced security protocols to protect your data. Regular updates and patches ensure that the platform remains secure against emerging threats.

**Compliance with Industry Standards:**
Compliance with industry standards such as GDPR, HIPAA, and ISO is crucial. IBM Concert is designed to meet these requirements, providing peace of mind for businesses in regulated industries.

**Best Practices for Maintaining Security:**
- **Regular Audits:** Conduct regular security audits to identify and address vulnerabilities.
- **Access Control:** Implement strict access controls to ensure that only authorized users can access sensitive data.

## Case Studies and Success Stories

Real-world examples can provide valuable insights into the effectiveness of IBM Concert.

**Real-World Examples of IBM Concert in Action:**
Several companies have successfully implemented IBM Concert, reporting improved project management and team collaboration. These case studies highlight the tool's ability to adapt to various industries and business sizes.

**Impact on Project Management and Team Collaboration:**
IBM Concert has been shown to enhance project visibility, streamline workflows, and foster better team collaboration. These improvements lead to more successful project outcomes.

**Lessons Learned from Successful Implementations:**
Success stories often reveal best practices and pitfalls to avoid. By learning from these experiences, you can optimize your own implementation of IBM Concert.

## Conclusion

Adopting IBM Concert and following these best practices can significantly enhance your project management capabilities. From leveraging AI and machine learning to ensuring robust security measures, IBM Concert offers a comprehensive solution for modern project management.

**Recap of Best Practices:**
- Thorough planning and customization.
- Effective training and onboarding.
- Integration of AI and machine learning.
- Emphasis on user experience and security.

**Encouragement to Adopt IBM Concert:**
By embracing IBM Concert, you can elevate your project management to new heights, driving efficiency, collaboration, and success.

## Call to Action

Ready to transform your project management? Try IBM Concert today and experience the difference for yourself. Visit our [official website](https://www.ibm.com) for more information, tutorials, and to schedule a demo or consultation.

---

This blog post is designed to inform and inspire project managers, IT professionals, and business owners to adopt IBM Concert, leveraging its powerful features to achieve their project management goals.
```

# Congratulations! You built a multiagent workflow that writes engaging blog posts!


## Other experimentation avenues:
- create a new post by changing the topic.
- change and use other watsonx.ai models.
- create a completely new system with new agents and tasks.
- add a tool to one of your agents, learn more about [crewAI tools here](https://docs.crewai.com/core-concepts/Tools/) 


## Details about this lab.

- Author: Jorge Castañón, Client Engineering Americas Top Team
- References: 
    - First notebook from deeplearning.ai [course]("https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/") on multi-ai-agents is the core of the lab.
    - This [notebook]("https://github.com/yobens5/crewai/blob/main/Crewai%20%2B%20Watsonx%20with%20Llama%203.2%20vision.ipynb") by Yohan Bensoussan from IBM helped me connect crewAI with watsonx.ai
- What to try next:    
    - Try multimodal ai agents using last [notebook]("https://github.com/yobens5/crewai/blob/main/Crewai%20%2B%20Watsonx%20with%20Llama%203.2%20vision.ipynb") by Yohan and the [wx.ai documentation](https://www.ibm.com/think/tutorials/multimodal-ai-python-llama)